# 使用 Ragas 做概念验证阶段评估

使用 Ragas 对 RAG 做概念验证阶段的性能评估：

- 创建一个简易的 RAG
- 给出一个真实答案的数据集
- 通过 Ragas 和数据集，对 RAG 做性能评估
    - 针对 LLM 生成内容
        - faithfulness，回答忠实性，生成答案的事实准确性
        - answer relevancy，答案相关性，生成的答案与问题的相关程度
    - 针对检索上下文内容
        - context precision，检索到的上下文精度
        - context recall，是否检索到回答问题所需的所有相关信息

## 创建简易的 RAG

In [4]:
%%time
%%capture

!pip install trafilatura
!pip install pandas
!pip install llama-index
!pip install llama-index-readers-web

CPU times: user 32.7 ms, sys: 30.8 ms, total: 63.5 ms
Wall time: 10.4 s


### 加载文档

In [6]:
%%time

from llama_index.readers.web import TrafilaturaWebReader

item="北海公园"

documents = TrafilaturaWebReader().load_data(
    [ f"https://baike.baidu.com/item/{item}" ]
)

documents[0].text[:500]

CPU times: user 108 ms, sys: 6.95 ms, total: 115 ms
Wall time: 163 ms


'北海公园（Beihai Park），位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海，全园占地68.2万平方米（其中水域面积38.9万平方米，陆地面积29.3万平方米），1925年开放为公园。 [3] [28]\n2023年4月，北海公园入选第二批国家级文明旅游示范单位公示名单。 [37]\n- 中文名\n- 北海公园\n- 外文名\n- Beihai Park\n- 地理位置\n- 北京市西城区文津街1号\n- 气候条件\n- 温带季风气候\n- 开放时间\n- 4月1日-10月31日 06:00-21:00；11月1日-次年3月31日 06:30-20:00 [1]\n- 景点级别\n- AAAA级\n- 门票价格\n- 旺季10元，淡季5元\n- 占地面积\n- 682000 m²（其中水面积38.9万平方米，陆地面积29.3万平方米）\n- 著名景点\n- 琼华岛、永安寺、北海白塔\n北海公园原为辽燕京城东北一片湖泊，名金海。中有小岛名瑶屿，辽在此建“瑶屿行宫”。金大定三年至十九年（1163年—1179年），金世宗仿照北宋汴梁（今河南开封）艮岳园，建琼华岛，并从“艮岳”御苑运来大量太湖石砌成假山岩洞，'

### 设置 LLM、嵌入和分块参数

In [ ]:
%%time
%%capture

!pip install llama-index-llms-openai-like
!pip install llama-index-embeddings-ollama

In [8]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.chunk_size = 128
Settings.chunk_overlap = 10

CPU times: user 174 ms, sys: 15.4 ms, total: 189 ms
Wall time: 189 ms


### 创建索引和查询引擎

In [17]:
%%time

from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)
query_engine = vector_index.as_query_engine(streaming=True)

CPU times: user 379 ms, sys: 17 ms, total: 396 ms
Wall time: 5.99 s


In [38]:
%%time

response = query_engine.query("北海公园在哪里？")
response.print_response_stream()
print()

北海公园位于北京市西城区文津街1号。
CPU times: user 35.8 ms, sys: 6.1 ms, total: 41.9 ms
Wall time: 534 ms


## 创建测试用数据集

In [21]:
%%time
%%capture

!pip install datasets

CPU times: user 41.1 ms, sys: 7.68 ms, total: 48.8 ms
Wall time: 9.12 s


In [22]:
%%time

from datasets import Dataset 

data_samples = {
    'question': ['北海公园在哪里？', ],
    'contexts' : ['北海公园（Beihai Park），位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海，全园占地68.2万平方米（其中水域面积38.9万平方米，陆地面积29.3万平方米），1925年开放为公园。'],
    'ground_truth': ['北海公园位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海。'],
    
}

CPU times: user 184 ms, sys: 28.7 ms, total: 213 ms
Wall time: 250 ms


In [23]:
%%time

import pandas as pd

# 创建Pandas DataFrame
df = pd.DataFrame(data_samples)

df.head()

CPU times: user 666 μs, sys: 104 μs, total: 770 μs
Wall time: 772 μs


,question,contexts,ground_truth
0,北海公园在哪里？,北海公园（Beihai Park），位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北...,北海公园位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海。


## 使用 Ragas 评估

### 设置评估用 LLM

In [27]:
%%time

evaluator_llm = OpenAILike(
    model="gpt-3.5-turbo", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

CPU times: user 485 μs, sys: 76 μs, total: 561 μs
Wall time: 565 μs


### 使用 Ragas

In [26]:
%%time
%%capture

!pip install ragas

CPU times: user 31.2 ms, sys: 7.93 ms, total: 39.1 ms
Wall time: 7.33 s


In [28]:
%%time

from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.metrics.critique import harmfulness

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    harmfulness,
]

CPU times: user 332 ms, sys: 35 ms, total: 367 ms
Wall time: 375 ms


In [34]:
%%time

from ragas.integrations.llama_index import evaluate

result = evaluate(
    query_engine=vector_index.as_query_engine(),
    metrics=metrics,
    dataset=data_samples,
    llm=evaluator_llm,
    embeddings=Settings.embed_model,
)

Running Query Engine:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

n values greater than 1 not support for LlamaIndex LLMs


CPU times: user 153 ms, sys: 21.3 ms, total: 174 ms
Wall time: 10.4 s


In [35]:
result.to_pandas()

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall,harmfulness
0,北海公园在哪里？,[北海公园（Beihai Park），位于北京市西城区文津街1号，东邻景山公园，南濒中南海，...,北海公园位于北京市西城区文津街1号。,北海公园位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海。,1.0,0.550745,1.0,1.0,0


In [36]:
result.scores.to_pandas()

,faithfulness,answer_relevancy,context_precision,context_recall,harmfulness
0,1.0,0.550745,1.0,1.0,0
